In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#export
from exp.nb_00 import *
def get_data():
    path = datasets.download_data(MNIST_URL, ext ='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train),(x_valid, y_valid),_) = pickle.load(f, encoding = 'latin-1')
    return map(tensor, (x_train,y_train, x_valid, y_valid))

def normalize(x,m,s):
    return (x-m)/s

???


In [3]:
x_train,y_train,x_valid,y_valid = get_data()


In [4]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [5]:
x_train = normalize(x_train,train_mean,train_std)
x_valid = normalize(x_valid,train_mean,train_std)


In [6]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean, train_std

(tensor(-7.6999e-06), tensor(1.))

In [7]:
 #export
def test_near_zero(a,tol = 1e-3): assert a.abs()<tol, f"Near zero:{a}"

In [8]:
test_near_zero(x_train.mean())
test_near_zero(x_train.std() - 1)

In [9]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

In [10]:
#hidden layers
nh = 50

In [11]:
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [12]:
test_near_zero(w1.mean())
test_near_zero(w1.std() - 1/math.sqrt(m))

In [13]:
x_valid.mean(),x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [14]:
def lin(x,w,b):return x@w+b

In [15]:
t = lin(x_valid,w1,b1)

In [16]:
t.mean(),t.std()

(tensor(0.1684), tensor(0.9449))

In [17]:
#Fixup Initialization

In [18]:
def relu(x):return x.clamp_min(0.)

In [19]:
t = relu(lin(x_valid,w1,b1))

In [20]:
t.mean(),t.std()

(tensor(0.4655), tensor(0.6000))

In [21]:
w1 = torch.randn(m,nh)*math.sqrt(2/m)


In [22]:
w1.mean(),w1.std()

(tensor(0.0003), tensor(0.0504))

In [23]:
t = relu(lin(x_valid,w1,b1))

In [24]:
t.mean(),t.std()

(tensor(0.5714), tensor(0.8168))

In [25]:
#export
from torch.nn import init

In [26]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1,mode='fan_out')#fan_in or fan_out divided by m or nh
t = relu(lin(x_valid,w1,b1))

In [27]:
t.mean(),t.std()

(tensor(0.4631), tensor(0.7242))

In [28]:
w1.shape

torch.Size([784, 50])

In [29]:
torch.nn.Linear(m,nh).weight.shape# also do a transpose

torch.Size([50, 784])

In [30]:
def relu(x):return x.clamp_min(0.) - 0.5

In [31]:
t = relu(lin(x_valid,w1,b1))

In [32]:
t.mean(),t.std()

(tensor(-0.0369), tensor(0.7242))

In [33]:
def model(xb):
    l1 = lin(xb,w1,b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [34]:
%timeit -n 10 _ = model(x_valid)

7.14 ms ± 86.5 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [35]:
assert model(x_valid).shape == torch.Size([x_valid.shape[0], 1])

In [36]:
model(x_valid).shape

torch.Size([10000, 1])

In [37]:
def mse(output, targ): return (output.squeeze()-targ).pow(2).mean()

In [38]:
y_train, y_valid = y_train.float(),y_valid.float()

In [39]:
preds = model(x_train)

In [40]:
preds.shape

torch.Size([50000, 1])

In [41]:
y_train[0]

tensor(5.)

In [42]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [43]:
def relu_grad(inp,out):
    inp.g = (inp>0).float()*out.g

In [44]:
def lin_grad(inp,out,w,b):
    inp.g = out.g@w.t()
    w.g = (inp.unsqueeze(-1)*out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [45]:
def forward_and_backward(inp, targ):
     # forward pass:\n",
        l1 = inp @ w1 + b1
        l2 = relu(l1)
        out = l2 @ w2 + b2
        # we don't actually need the loss in backward!\n",
        loss = mse(out, targ)
        # backward pass:\n",
        mse_grad(out, targ)
        lin_grad(l2, out, w2, b2)
        relu_grad(l1, l2)
        lin_grad(inp, l1, w1, b1)
    

In [46]:
forward_and_backward(x_train,y_train)

In [50]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

In [47]:
init?

In [48]:
torch.nn.modules.conv._ConvNd.reset_parameters??

### Refactor model 重构模型

In [49]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float()*self.out.g

In [51]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [52]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [53]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [54]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [55]:
%time loss = model(x_train, y_train)

CPU times: user 142 ms, sys: 0 ns, total: 142 ms
Wall time: 35.5 ms


In [56]:
%time model.backward()

CPU times: user 5.05 s, sys: 2.09 s, total: 7.13 s
Wall time: 1.95 s


In [57]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

In [58]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [59]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [60]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [61]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [62]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [63]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [64]:
%time loss = model(x_train, y_train)

CPU times: user 138 ms, sys: 0 ns, total: 138 ms
Wall time: 34.8 ms


In [65]:
%time model.backward()


CPU times: user 330 ms, sys: 51.3 ms, total: 381 ms
Wall time: 99.9 ms


In [69]:
!python notebook2script.py 01_AI.ipynb

Converted 01_AI.ipynb to exp/nb_01.py
